# Prepare the Web Driver

The tutorial is from https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d

In [1]:
from selenium import webdriver
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
# If this error occurs: WebDriverException: Message: 'Webdrivers' executable may have wrong permissions.
## See this post: https://stackoverflow.com/questions/47148872/webdrivers-executable-may-have-wrong-permissions-please-see-https-sites-goo

DRIVER_PATH = '/home/shin/Bangkit Academy/CAPSTONE PROJECT/chromedriver_linux64/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

# Function: Scrap Images URLs

In [2]:
import time

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

# Function: Download Images from URLs

In [3]:
from PIL import Image
import os
import io
import requests
import hashlib

def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

# Webscrap

In [4]:
input_filename = 'nama_tumbuhan.txt' # file containing list of plants to be searched
target_folder = 'images/'   # folder to save the images
max_images_per_keyword = 3

In [5]:
f = open(input_filename, 'r')
lines = f.readlines()
for line in lines:
    line = line.strip()
    keyword = 'daun ' + line
    folder_to_save = os.path.join(target_folder, line.replace(' ', '_'))
    try:
        os.mkdir(folder_to_save)
    except:
        pass
    urls = fetch_image_urls(query=keyword, max_links_to_fetch=max_images_per_keyword, wd=wd, sleep_between_interactions=1)

    # save images from URLs
    for url in urls:
        persist_image(folder_path=folder_to_save, url=url)

Found: 100 search results. Extracting links from 0:100
Found: 3 image links, done!
SUCCESS - saved https://www.nugrahaorchid.com/wp-content/uploads/2019/08/daun-anggrek-bulan.jpg - as images/anggrek/d65df992d3.jpg
SUCCESS - saved https://id-test-11.slatic.net/p/563957f385fd899e1cdd708d6db627cc.jpg_720x720q80.jpg_.webp - as images/anggrek/9f56a36e52.jpg
SUCCESS - saved https://i.ytimg.com/vi/NkGMNFOr10E/maxresdefault.jpg - as images/anggrek/f57a5e40f1.jpg
Found: 100 search results. Extracting links from 0:100
Found: 3 image links, done!
SUCCESS - saved https://id-test-11.slatic.net/p/6bd54a65e76bc64419b031dd3b38b224.jpg_720x720q80.jpg_.webp - as images/mawar/9615165327.jpg
SUCCESS - saved https://images.tokopedia.net/img/cache/700/product-1/2017/1/31/615970/615970_6cbbcb41-0334-4a89-98e5-34a32c9d9646.jpg - as images/mawar/bd1acc6fba.jpg
SUCCESS - saved https://ae01.alicdn.com/kf/HTB1upYBOVXXXXX4aFXXq6xXFXXXT.jpg_q50.jpg - as images/mawar/513abe5fcc.jpg
Found: 100 search results. Extract

# Close the Web Driver

In [ ]:
wd.close()